# TSF using Neural Networks


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
# Imports

import numpy as np
import pandas as pd
import pandas.util.testing as tm
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('dark_background')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  """


In [ ]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.convolutional_recurrent import ConvLSTM2D
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
data = pd.read_csv('/content/gdrive/MyDrive/AirPassengers.csv')
data = data.rename(columns={'#Passengers': 'Passengers'})
data = data["Passengers"]

In [ ]:
data.head()

0    112
1    118
2    132
3    129
4    121
Name: Passengers, dtype: int64

In [ ]:
df = data.values
df = df.astype('float32') 
df

array([112., 118., 132., 129., 121., 135., 148., 148., 136., 119., 104.,
       118., 115., 126., 141., 135., 125., 149., 170., 170., 158., 133.,
       114., 140., 145., 150., 178., 163., 172., 178., 199., 199., 184.,
       162., 146., 166., 171., 180., 193., 181., 183., 218., 230., 242.,
       209., 191., 172., 194., 196., 196., 236., 235., 229., 243., 264.,
       272., 237., 211., 180., 201., 204., 188., 235., 227., 234., 264.,
       302., 293., 259., 229., 203., 229., 242., 233., 267., 269., 270.,
       315., 364., 347., 312., 274., 237., 278., 284., 277., 317., 313.,
       318., 374., 413., 405., 355., 306., 271., 306., 315., 301., 356.,
       348., 355., 422., 465., 467., 404., 347., 305., 336., 340., 318.,
       362., 348., 363., 435., 491., 505., 404., 359., 310., 337., 360.,
       342., 406., 396., 420., 472., 548., 559., 463., 407., 362., 405.,
       417., 391., 419., 461., 472., 535., 622., 606., 508., 461., 390.,
       432.], dtype=float32)

In [ ]:
scaler = MinMaxScaler()
df = df.reshape(-1, 1)  # reshaping for scaling
scaler = MinMaxScaler() # optional for single feature
df = scaler.fit_transform(df)
#df

In [ ]:
train_size = int(len(df) * 0.66)  #seq is imp
test_size = len(df) - train_size
train, test = df[0:train_size,:], df[train_size:len(df),:]
train_size,test_size

(95, 49)

In [ ]:
# Data transformation, 1,2,3,4,5->6 ,2,3,4,5,6,->7...
# Making a window, predicting a val witht the help of prev values in the window
def data_transf(data, win_size):
    x = []
    y = []

    for i in range(len(data)-win_size-1):
        window = df[i:(i+win_size), 0]
        x.append(window)
        y.append(df[i+ win_size, 0])
        
    return np.array(x),np.array(y)

In [ ]:
win_size = 7
X_train, y_train = data_transf(train, win_size)
X_test, y_test = data_transf(test, win_size)
X_train.shape,X_test.shape

((87, 7), (41, 7))

In [ ]:
X_train[1],train[0:10]

(array([0.02702703, 0.05405405, 0.04826255, 0.03281853, 0.05984557,
        0.08494207, 0.08494207], dtype=float32), array([[0.01544401],
        [0.02702703],
        [0.05405405],
        [0.04826255],
        [0.03281853],
        [0.05984557],
        [0.08494207],
        [0.08494207],
        [0.06177607],
        [0.02895753]], dtype=float32))

In [ ]:
# Model
model = Sequential()
model.add(Dense(64, input_dim= win_size, activation='relu')) #12
model.add(Dense(32, activation='relu'))  #8
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
print(model.summary()) 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                512       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,625
Trainable params: 2,625
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test),verbose=2, epochs=100)

Epoch 1/100
3/3 - 12s - loss: 0.0920 - acc: 0.0115 - val_loss: 0.0141 - val_acc: 0.0244
Epoch 2/100
3/3 - 0s - loss: 0.0530 - acc: 0.0115 - val_loss: 0.0067 - val_acc: 0.0244
Epoch 3/100
3/3 - 0s - loss: 0.0297 - acc: 0.0115 - val_loss: 0.0034 - val_acc: 0.0244
Epoch 4/100
3/3 - 0s - loss: 0.0157 - acc: 0.0115 - val_loss: 0.0026 - val_acc: 0.0244
Epoch 5/100
3/3 - 0s - loss: 0.0090 - acc: 0.0115 - val_loss: 0.0033 - val_acc: 0.0244
Epoch 6/100
3/3 - 0s - loss: 0.0072 - acc: 0.0115 - val_loss: 0.0048 - val_acc: 0.0244
Epoch 7/100
3/3 - 0s - loss: 0.0074 - acc: 0.0115 - val_loss: 0.0060 - val_acc: 0.0244
Epoch 8/100
3/3 - 0s - loss: 0.0082 - acc: 0.0115 - val_loss: 0.0064 - val_acc: 0.0244
Epoch 9/100
3/3 - 0s - loss: 0.0082 - acc: 0.0115 - val_loss: 0.0058 - val_acc: 0.0244
Epoch 10/100
3/3 - 0s - loss: 0.0074 - acc: 0.0115 - val_loss: 0.0047 - val_acc: 0.0244
Epoch 11/100
3/3 - 0s - loss: 0.0063 - acc: 0.0115 - val_loss: 0.0036 - val_acc: 0.0244
Epoch 12/100
3/3 - 0s - loss: 0.0056 - a

In [ ]:
pred_tr = model.predict(X_train)
pred_te = model.predict(X_test)

In [ ]:
# Inverse transformation to bring actual values

pred_tr = scaler.inverse_transform(pred_tr)
ytr_act = scaler.inverse_transform([y_train])
pred_te = scaler.inverse_transform(pred_te)
yte_act = scaler.inverse_transform([y_test])

In [ ]:
train_rmse = math.sqrt(mean_squared_error(ytr_act[0], pred_tr[:,0]))
test_rmse = math.sqrt(mean_squared_error(yte_act[0], pred_te[:,0]))
train_rmse,test_rmse

(19.806734931482907, 15.065882909982948)

In [ ]:
# Unfloding data for plotting
predtr_plot = np.empty_like(df)
predtr_plot[:, :] = np.nan  # empty array with nan values
predtr_plot[win_size:len(pred_tr)+win_size, :] = pred_tr

predtest_plot = np.empty_like(df)
predtest_plot[:, :] = np.nan
predtest_plot[len(pred_tr)+(win_size*2)+1:len(df)-1, :] = pred_te